In [ ]:
import numpy as np
import glob
import imageio.v2 as imageio
import vedo
vedo.settings.default_backend= 'vtk'
from vedo import Plane
import brainrender
from brainrender import Scene
from brainglobe_atlasapi import BrainGlobeAtlas
import matplotlib.pyplot as plt
import re
from pathlib import Path


In [ ]:
import random
from pathlib import Path
import itertools

import numpy as np
from myterial import orange
from rich import print

from brainrender import Scene
from brainrender.actors import Points
from matplotlib import colors as mcolors
from tqdm import tqdm
import pandas as pd
data_path = Path("C:/Microscope_images/processed/becalia_rabies_barseq/BRAC8498.3e/ara_spots")
ara_barcode_spots = pd.read_pickle(data_path / "ara_barcode_spots.pkl")
ara_starters = pd.read_pickle(data_path / "ara_starter_cells.pkl")


def read_in_BrainJ_cells(cells, regions, acronyms, not_in=None):
    if len(regions) > 0:
        cells = cells[cells['area_id'].isin(regions)]
    if len(acronyms) > 0:
        cells = cells[cells['area_acronym'].isin(acronyms)]
    if not_in is not None:
        cells = cells[~cells['area_acronym'].isin(not_in)]
    Z = cells['ara_z'].tolist()
    Y = cells['ara_y'].tolist()
    X = cells['ara_x'].tolist()
    
    AtlasRes = 1000
    X = [element * AtlasRes for element in X]
    Y = [element * AtlasRes for element in Y]
    Z = [element * AtlasRes for element in Z]
    
    if len(X) == 0:
        return np.empty((0, 3))
    
    pts = [[x, y, z] for x, y, z in zip(X, Y, Z)]
    return np.vstack(pts)

def get_cycled_colors(n):
    colors = list(mcolors.CSS4_COLORS.keys())
    random.shuffle(colors)
    return [colors[i % len(colors)] for i in range(n)]

# Get unique barcodes present in both DataFrames
common_barcodes = set(ara_barcode_spots['barcode'].unique()) & set(ara_starters['main_barcode'].unique())

# Filter DataFrames to only include common barcodes
ara_barcode_spots = ara_barcode_spots[ara_barcode_spots['barcode'].isin(common_barcodes)]
ara_starters = ara_starters[ara_starters['main_barcode'].isin(common_barcodes)]

# Function to render specific barcodes
def render_selected_barcodes(selected_barcodes=None):
    if selected_barcodes is None:
        # Get all barcodes
        selected_barcodes = common_barcodes
    elif type(selected_barcodes) == int:
        selected_barcodes = set(itertools.islice(common_barcodes, selected_barcodes))
    unique_barcodes = list(set(selected_barcodes) & common_barcodes)
    colors = get_cycled_colors(len(unique_barcodes))
    barcode_to_color = dict(zip(unique_barcodes, colors))

    # Group cells by barcode and color
    for barcode in tqdm(unique_barcodes, desc="Processing barcodes", unit="barcode"):
        # Get points for current barcode
        rabies_cells = read_in_BrainJ_cells(
            ara_starters[(ara_starters["starter"]==False) & (ara_starters["main_barcode"]==barcode)],
            [],
            [],
            ["outside"]
        )
        
        starter_cells = read_in_BrainJ_cells(
            ara_starters[(ara_starters["starter"]==True) & (ara_starters["main_barcode"]==barcode)],
            [],
            [],
            ["outside"]
        )
        
        rabies_spots = read_in_BrainJ_cells(
            ara_barcode_spots[ara_barcode_spots["barcode"]==barcode],
            [],
            [],
            ["outside"]
        )

        color = barcode_to_color[barcode]

        if rabies_cells.size > 0:
            scene.add(Points(
                rabies_cells,
                name=f"rabies_cells_{barcode}",
                colors=color,
                alpha=0.2,
                radius=10
            ))
        if starter_cells.size > 0:
            scene.add(Points(
                starter_cells,
                name=f"starter_cells_{barcode}",
                colors=color,
                alpha=0.4,
                radius=20
            ))
        if rabies_spots.size > 0:
            scene.add(Points(
                rabies_spots,
                name=f"rabies_spots_{barcode}",
                colors=color,
                alpha=1,
                radius=2
            ))

In [ ]:
infected_areas = [   
    ### Auditory primary
    'AUDp1',
    'AUDp2/3',
    'AUDp4',
    'AUDp5',
    'AUDp6a',
    'AUDp6b',
    ### Auditory posterior
    'AUDpo1',
    'AUDpo2/3',
    'AUDpo4',
    'AUDpo5',
    'AUDpo6a',
    'AUDpo6b',
    ### Auditory ventral
    'AUDv2/3',
    'AUDv4',
    'AUDv5',
    'AUDv6a',
    'AUDv6b',
    ### Thalamus
    'IGL',
    'IntG',
    'LGd-co',
    'LGd-sh',
    'LGv',
    ### Retrosplenial lateral agranular
    'RSPagl1',
    'RSPagl2/3',
    'RSPagl5',
    'RSPagl6a',
    'RSPagl6b',
    ### Retrosplenial dorsal
    'RSPd1',
    'RSPd2/3',
    'RSPd5',
    'RSPd6a',
    'RSPd6b',
    ### Retrosplenial ventral
    'RSPv1',
    'RSPv2/3',
    'RSPv5',
    'RSPv6a',
    ### Visual antero-lateral
    'VISal1',
    'VISal2/3',
    'VISal4',
    'VISal5',
    'VISal6a',
    'VISal6b',
    ### Visual lateral
    'VISl1',
    'VISl2/3',
    'VISl4',
    'VISl5',
    'VISl6a',
    'VISl6b',
    ### Visual laterointermediate
    'VISli1',
    'VISli2/3',
    'VISli4',
    'VISli5',
    'VISli6a',
    'VISli6b',
    ### Visual primary
    'VISp1',
    'VISp2/3',
    'VISp4',
    'VISp5',
    'VISp6a',
    'VISp6b',
    ### Visual posteromedial
    'VISpm1',
    'VISpm2/3',
    'VISpm4',
    'VISpm5',
    'VISpm6a',
    'VISpm6b',
]

In [ ]:
import numpy as np
import matplotlib.colors as mcolors


def generate_shades(base_color, num_shades):
    base_rgb = mcolors.to_rgb(base_color)
    # Adjust the range of alpha values to create a more extreme gradient
    return [mcolors.to_hex((base_rgb[0] * alpha, base_rgb[1] * alpha, base_rgb[2] * alpha)) for alpha in np.linspace(1, 0.1, num_shades)]

# Define the broad areas and their base colors
large_actors = {
    "AUDpo" : "brown",
    "VISp" : "blue",
    "VISpl" : "lightgreen",
    "VISl" : "mediumspringgreen",
    "VISal" : "lime",
    "VISpm" : "deepskyblue",
    "VISli" : "cyan",
    "RSP" : "deeppink",
    "TEa" : "gold",
    'TH' : "blueviolet",
    "AUDd" : "yellow",
    "AUDv" : "sandybrown",
    "AUDp" : "orange",
}

# Sub-areas that need to be shaded
infected_areas = {   
    ### Auditory primary
    'AUDp1': "AUDp",
    'AUDp2/3': "AUDp",
    'AUDp4': "AUDp",
    'AUDp5': "AUDp",
    'AUDp6a': "AUDp",
    'AUDp6b': "AUDp",
    ### Auditory posterior
    'AUDpo1': "AUDpo",
    'AUDpo2/3': "AUDpo",
    'AUDpo4': "AUDpo",
    'AUDpo5': "AUDpo",
    'AUDpo6a': "AUDpo",
    'AUDpo6b': "AUDpo",
    ### Auditory ventral
    'AUDv2/3': "AUDv",
    'AUDv4': "AUDv",
    'AUDv5': "AUDv",
    'AUDv6a': "AUDv",
    'AUDv6b': "AUDv",
    ### Thalamus
    'IGL': "TH",
    "LGd-ip" : "TH",
    "LGd-sh" : "TH",
    'LGd-co': "TH",
    'MG': "TH",
    'MGd': "TH",
    'MGv': "TH",
    "LAT" : "TH",
    'IntG': "TH",
    'LGd': "TH",
    'LGv': "TH",
    "VENT" : "TH",
    "PP" : "TH",
    "PIL" : "TH",
    "VPM" : "TH",
    "VPMpc" : "TH",
    "VM" : "TH",
    "POL" : "TH",
    ### Retrosplenial lateral agranular
    'RSPagl1': "RSP",
    'RSPagl2/3': "RSP",
    'RSPagl5': "RSP",
    'RSPagl6a': "RSP",
    'RSPagl6b': "RSP",
    ### Retrosplenial dorsal
    'RSPd1': "RSP",
    'RSPd2/3': "RSP",
    'RSPd5': "RSP",
    'RSPd6a': "RSP",
    'RSPd6b': "RSP",
    ### Retrosplenial ventral
    'RSPv1': "RSP",
    'RSPv2/3': "RSP",
    'RSPv5': "RSP",
    'RSPv6a': "RSP",
    ### Visual antero-lateral
    'VISal1': "VISal",
    'VISal2/3': "VISal",
    'VISal4': "VISal",
    'VISal5': "VISal",
    'VISal6a': "VISal",
    'VISal6b': "VISal",
    ### Visual lateral
    'VISl1': "VISl",
    'VISl2/3': "VISl",
    'VISl4': "VISl",
    'VISl5': "VISl",
    'VISl6a': "VISl",
    'VISl6b': "VISl",
    ### Visual laterointermediate
    'VISli1': "VISli",
    'VISli2/3': "VISli",
    'VISli4': "VISli",
    'VISli5': "VISli",
    'VISli6a': "VISli",
    'VISli6b': "VISli",
    ### Visual primary
    'VISp1': "VISp",
    'VISp2/3': "VISp",
    'VISp4': "VISp",
    'VISp5': "VISp",
    'VISp6a': "VISp",
    'VISp6b': "VISp",
    ### Visual posteromedial
    'VISpm1': "VISpm",
    'VISpm2/3': "VISpm",
    'VISpm4': "VISpm",
    'VISpm5': "VISpm",
    'VISpm6a': "VISpm",
    'VISpm6b': "VISpm",
    ### TEa
    'TEa1': "TEa",
    'TEa2/3': "TEa",
    'TEa4': "TEa",
    'TEa5': "TEa",
    'TEa6a': "TEa",
    'TEa6b': "TEa",
}

# Create the gradient colors for the infected areas
infected_colors = {}
for base_area, base_color in large_actors.items():
    num_shades = len([key for key in infected_areas if infected_areas[key] == base_area])
    shades = generate_shades(base_color, num_shades)
    for area in [key for key in infected_areas if infected_areas[key] == base_area]:
        infected_colors[area] = shades.pop(0)

#import ace_tools as tools; tools.display_dataframe_to_user(name="Infected Areas Colors", dataframe=pd.DataFrame.from_dict(infected_colors, orient='index', columns=['Color']))

infected_colors



In [ ]:
unimportant_areas = [
    'APN',
    ###
    'CA1',
    'CA3',
    'DG-mo',
    'DG-po',
    'DG-sg',
    ### Ectorhinal
    'ECT5',
    'ECT6a',
    ### Entorhinal
    'ENTl5',
    ###
    'LP',
    'MB',
    'MGd',
    'MGv',
    'MRN',
    'ND',
    'PAG',
    ###
    'PERI6a',
    'PERI6b',
    ###
    'POL',
    ###
    'PoT',
    ###
    'SCig',
    'SCop',
    'SCsg',
    'SCzo',
    'SGN',
    ### Temporal association
    'TEa1',
    'TEa2/3',
    'TEa4',
    'TEa5',
    'TEa6a',
    'TEa6b',
    ###
    'alv',
    'ar',
    'bsc',
    'cing',
    'dhc',
    'ec',
    'fiber tracts',
    'fp',
    'or',
    'scwm'
]

In [ ]:
import brainrender
# Initialize brainrender scene
brainrender.settings.BACKGROUND_COLOR = [
    0.12,
    0.12,
    0.12,
]  # change rendering background color
brainrender.settings.WHOLE_SCREEN = (
    True  # make the rendering window be smaller
)
brainrender.settings.SHOW_AXES = False # turn off the axes display
brainrender.settings.ROOT_ALPHA= (0.1)
brainrender.settings.ROOT_COLOR = "white"
brainrender.settings.DEFAULT_ATLAS = "allen_mouse_10um"

tab20 = plt.colormaps["tab20"]

from vedo import Plotter
# Subclass the Plotter to add custom keybindings
class CustomPlotter(Plotter):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def keypress(self, key):
        if key == 'k':
            self.camera.Azimuth(10)
        elif key == 'j':
            self.camera.Elevation(10)
        elif key == 'h':
            self.camera.Roll(10)
        super().keypress(key)
        self.render()

# Create a brainrender scene with the custom plotter
#scene = Scene(plotter_class=CustomPlotter)
scene = Scene()

large_actors = {
    "AUDpo" : "brown",
    "VISp" : "blue",
    "VISpl" : "lightgreen",
    "VISl" : "mediumspringgreen",
    "VISal" : "lime",
    "VISpm" : "deepskyblue",
    "VISli" : "cyan",
    "RSP" : "deeppink",
    "TEa" : "gold",
    'TH' : "blueviolet",
    "AUDd" : "yellow",
    "AUDv" : "sandybrown",
    "AUDp" : "orange",
}

large_non_cortical_actors = [
    "HPF",
    "ENT",
    "PERI",
    "ECT",
    "BS",
    "fiber tracts",
    "OLF",
    "CTXsp"
]
all_actors = ara_barcode_spots["area_acronym"].unique().astype(str)
all_actors = all_actors.tolist()
all_actors.remove("root")
brainreg_positions1 = [
    np.array([8621.842 , 397.942 , 5706.5796], dtype="float32"),
    np.array([9575.921 , 214.3228, 5678.3687], dtype="float32")
]
normal_vectors1 = [(-1.8919415, -0.12186934, 0.03463937), (1.8919415, 0.12186943, -0.03463934)]

atlas = BrainGlobeAtlas("allen_mouse_10um", check_latest=False)

# Add infected areas
added_actors = []
for i, actor in enumerate(infected_colors.items()):
    #print(actor[0])
    #print(added_actor._mesh)
    added_actor = scene.add_brain_region(actor[0], alpha=0.1, color= actor[1], hemisphere="left", silhouette=False)
    #scene.add_silhouette(added_actor, lw=0.00001, color=actor[1])


    added_actors.append(added_actor)

#for i, actor in enumerate(unimportant_areas):
    #added_actor = scene.add_brain_region(actor, alpha=0.001, color= tab20(i % 20)[:3], hemisphere="left")
    #print(actor)
    #print(added_actor._mesh)
    #scene.add_silhouette(added_actor, lw=0.00001, color="grey")#tab20(i % 20)[:3])

    #added_actors.append(added_actor)

for i, actor in enumerate(large_actors.items()):
    added_actor = scene.add_brain_region(actor[0], alpha=0.001, color= actor[1])# hemisphere="left")
    #print(actor[0])
    #print(added_actor._mesh)
    scene.add_silhouette(added_actor, lw=1, color=actor[1])
    #scene.add_label(added_actor, actor,color= tab20(i % 20)[:3])

    added_actors.append(added_actor)

for i, actor in enumerate(large_non_cortical_actors):
    added_actor = scene.add_brain_region(actor, alpha=0.05, color= "grey",)# hemisphere="left")
    #print(actor)
    #print(added_actor._mesh)
    scene.add_silhouette(added_actor, lw=0.0001, color="grey")
    #scene.add_label(added_actor, actor,color= tab20(i % 20)[:3])

    added_actors.append(added_actor)

selected_barcodes = ["AGACTATGCTAAGC", "TTAATACGGGCTTT", "TACCATTAAGGCTG"]  # Replace with actual barcodes
render_selected_barcodes(200) #selected_barcodes)
plane = Plane(pos=brainreg_positions1[0], s=(10000,5000), normal=normal_vectors1[1]) 
plane2 = Plane(pos=brainreg_positions1[1], s=(10000,5000), normal=normal_vectors1[0])
scene.slice(plane, actors=added_actors)
scene.slice(plane2, actors=added_actors, close_actors=True)






# Initialize a list to store camera information
camera_info = []

# Create a function to print the camera position, focal point, distance, and zoom
def print_camera_info(event):
    cam = scene.plotter.camera
    cam_pos = cam.GetPosition()
    focal_point = cam.GetFocalPoint()
    distance = cam.GetDistance()
    zoom = scene.plotter.camera.GetParallelScale() if scene.plotter.camera.GetParallelProjection() else scene.plotter.camera.GetViewAngle()
    
    info = {
        "position_x": cam_pos[0],
        "position_y": cam_pos[1],
        "position_z": cam_pos[2],
        "focal_point_x": focal_point[0],
        "focal_point_y": focal_point[1],
        "focal_point_z": focal_point[2],
        "distance": distance,
        "zoom": zoom
    }
    
    # Store the information in the list
    camera_info.append(info)
    
    # Print all information in one line
    print(f"Position: {cam_pos}, Focal Point: {focal_point}, Distance: {distance}, Zoom: {zoom}")

# Add a callback to the plotter to print the camera information
scene.plotter.add_callback('keypress', print_camera_info)

if False:
    # Define custom keybindings
    def rotate_x(event):
        scene.plotter.camera.Azimuth(10)
        scene.plotter.render()

    def rotate_y(event):
        scene.plotter.camera.Elevation(10)
        scene.plotter.render()

    def rotate_z(event):
        scene.plotter.camera.Roll(10)
        scene.plotter.render()

    # Add custom keybindings to the plotter
    scene.plotter.add_callback('k', rotate_x)  # Rotate around x-axis when 'k' is pressed
    scene.plotter.add_callback('j', rotate_y)  # Rotate around y-axis when 'j' is pressed
    scene.plotter.add_callback('h', rotate_z)  # Rotate around z-axis when 'h' is pressed

# Render the scene with the interactive GUI
scene.render(interactive=True)


# Convert the list of camera information to a DataFrame
camera_df = pd.DataFrame(camera_info)

# Display the DataFrame
print("Stored Camera Information DataFrame:")
print(camera_df)

In [ ]:
# Initialize brainrender scene
scene = Scene()
tab20 = plt.colormaps["tab20"]

folder_path = 'C:/Microscope_data/BRAC8498.3e/overviews/'
# Create a list of all .tif files ending with .ome.tif_Coords.tif
tif_files = glob.glob(f'{folder_path}/*.ome.tif_Coords.tif')
#Order the files alphabetically
tif_files.sort()
# Select the first and last file
first_slice = tif_files[0]
last_slice = tif_files[-1]

rotate = False  

# Iterate over the list of files
for i, file_path in enumerate(tif_files):

    # Load the image
    img = imageio.imread(file_path)  # Adjust this path

    # img.shape is (3, height, width)
    # Let's pick three points: top-left corner, top-right corner, and bottom-left corner of the image
    p1 = np.array([img[0, 0, 0], img[1, 0, 0], img[2, 0, 0]])  # Top-left corner
    p2 = np.array([img[0, 0, -1], img[1, 0, -1], img[2, 0, -1]])  # Top-right corner
    p3 = np.array([img[0, -1, 0], img[1, -1, 0], img[2, -1, 0]])  # Bottom-left corner

    #Let's find the central point of the image too
    p4 = np.array(
                [img[0, img.shape[1] // 2, img.shape[2] // 2],
                 img[1, img.shape[1] // 2, img.shape[2] // 2],
                 img[2, img.shape[1] // 2, img.shape[2] // 2]])

    # Compute vectors
    v1 = p2 - p1
    v2 = p3 - p1

    # Compute the normal vector to the plane defined by v1 and v2
    normal_vector = np.cross(v1, v2)
    normal_vector = normal_vector / np.linalg.norm(normal_vector)  # Normalize the vector
    #normal_vector = (normal_vector[2], normal_vector[1], normal_vector[0])  # Adjust the order of the elements
    brainreg_position = (p4[0] * 1000, p4[1] * 1000, p4[2] * 1000)

    c = tab20(i % 20)[:3]

    # Add the plane to the scene
    plane = Plane(pos=brainreg_position, s=(10000, 5000), normal=tuple(normal_vector), c=c) #  
    scene.add(plane)

# Render the scene
scene.render()


In [ ]:
# 0 1 2
# straight down midline
# 0 2 1
# straight down midline but flipped direction
# 1 0 2
# flat across midline (brown up)
# 1 2 0
# correct plane, blue first
# 2 0 1
#crash
# 2 1 0
#correct plane, rotated 90 degrees, blue first


In [ ]:
import numpy as np
import imageio
from brainglobe_atlasapi import BrainGlobeAtlas

folder_path = 'C:/Microscope_data/BRAC8498.3e/overviews/'
# Create a list of all .tif files ending with .ome.tif_Coords.tif
tif_files = glob.glob(f'{folder_path}/*.ome.tif_Coords.tif')

# Iterate over the list of files
for i, file_path in enumerate(tif_files):
    test_path = file_path

def get_unique_actors_from_tif(file_path):
    """ Given a path to a .tif file, this function returns a list of unique actors in the image.

    Args:
        file_path (str): Path to the .tif file

    Returns:
        list: List of unique actors in the image
    """
    # Load the BrainGlobe Atlas
    bg_atlas = BrainGlobeAtlas("allen_mouse_25um", check_latest=False)
    
    # Load the image
    img = imageio.imread(file_path)
    
    # Assume img.shape is (3, height, width)
    height, width = img.shape[1], img.shape[2]
    
    # Calculate step sizes for the grid
    step_x = 1
    step_y = 1
    
    # Initialize an empty set for unique actors
    unique_actors = set()
    
    # Sample points in a 100x100 grid
    for i in range(0, height, step_y):
        for j in range(0, width, step_x):
            # Convert image coordinates to atlas coordinates
            # Assume that the conversion to atlas coordinates is similar to brainreg_position calculation
            x, y = j, i
            coord = (img[2, i, j] * 1000, img[1, i, j] * 1000, img[0, i, j] * 1000)
            # Query the atlas for the structure at this coordinate
            try:
                structure = bg_atlas.structure_from_coords(coord, microns=True, as_acronym=True,  hierarchy_lev=2)
            except:
                structure = None
            # Add the result to the set of unique actors
            if structure is not None:
                unique_actors.add(structure)
    
    # Convert the set to a list to return
    return list(unique_actors)
